In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 1,
    "ProductOffering": 1,
    "Facility": 1,
    "Parts": 1,
    "Warehouse": 1,
    "Supplier": 1,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "static_small_10_bulk_test_v3"

In [4]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)



Generated 13 operations
[{'action': 'create', 'type': 'schema', 'payload': {'node_id': '1', 'node_type': 'Parts', 'properties': {'id': '1', 'name': 'Name_6771', 'description': 'Description_8727', 'type': 'Type A', 'cost': 4835.28, 'importance': 2, 'expected_life': 2446, 'units_in_chain': 41, 'expiry': 191}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'id': '1-1', 'name': 'Name_3519', 'type': 'Type B', 'location': 'New York', 'max_capacity': 3678, 'operating_cost': 1616.1}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'quantity': 91, 'distance_from_warehouse': 772.62, 'transport_cost': 337.15, 'lead_time': 14}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'properties': {'id': '1-1-1', 'name': 'Name_2711', 'cost': 4608.79, '

In [5]:
bulk_create_payload = {
    "version": version,
    "action": "bulk_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": []
}

for op in create_ops:
    bulk_create_payload["payload"].append(op["payload"])

print(bulk_create_payload)

requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

{'version': 'static_small_10_bulk_test_v2', 'action': 'bulk_create', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '1', 'node_type': 'Parts', 'properties': {'id': '1', 'name': 'Name_6771', 'description': 'Description_8727', 'type': 'Type A', 'cost': 4835.28, 'importance': 2, 'expected_life': 2446, 'units_in_chain': 41, 'expiry': 191}}, {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'id': '1-1', 'name': 'Name_3519', 'type': 'Type B', 'location': 'New York', 'max_capacity': 3678, 'operating_cost': 1616.1}}, {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'quantity': 91, 'distance_from_warehouse': 772.62, 'transport_cost': 337.15, 'lead_time': 14}}, {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'properties': {'id': '1-1-1', 'name': 'Name_2711', 'cost': 4608.79, 'demand': 733}}, {'source_id': '1-1', 'target_id': '1-1-1', 'edge_type': 'FacilityToProductOfferings', 'properties': {'product_cost': 733.47, 'lead_time': 20, 'qu

<Response [200]>

In [6]:
update_cycles = 10
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 20 update operations
{'version': 'static_small_10_bulk_test_v2', 'action': 'bulk_update', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '1', 'node_type': 'Parts', 'payload': {'id': '1', 'name': 'Name_3277', 'description': 'Description_8727', 'type': 'Type B', 'cost': 4835.28, 'importance': 2, 'expected_life': 2446, 'units_in_chain': 41, 'expiry': 191}}, {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'payload': {'id': '1-1-1', 'name': 'Name_5487', 'cost': 4608.79, 'demand': 733}}, {'node_id': '1-1', 'node_type': 'Facility', 'payload': {'id': '1-1', 'name': 'Name_3519', 'type': 'Type B', 'location': 'New York', 'max_capacity': 3678, 'operating_cost': 8041.07}}, {'node_id': '1', 'node_type': 'Parts', 'payload': {'id': '1', 'name': 'Name_6684', 'description': 'Description_8727', 'type': 'Type A', 'cost': 8947.78, 'importance': 2, 'expected_life': 1148, 'units_in_chain': 41, 'expiry': 191}}, {'node_id': '1-1-1-1-1', 'node_type': 'BusinessUnit', 'payload': {'id': 

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)